In [9]:
# LOAD LIBRARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, AvgPool2D, BatchNormalization, Reshape
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import LearningRateScheduler
import matplotlib.pyplot as plt

In [10]:
# LOAD THE DATA
train = pd.read_csv('train.csv')

# PREPARE DATA FOR NEURAL NETWORK
Y_train = train["label"]
X_train = train.drop(labels = ["label"],axis = 1)
X_train = X_train / 255.0
X_train = X_train.values.reshape(-1,28,28,1)
Y_train = to_categorical(Y_train, num_classes = 10)

# GLOBAL VARIABLES
annealer = LearningRateScheduler(lambda x: 1e-3 * 0.95 ** x, verbose=0)
styles=[':','-.','--','-',':','-.','--','-',':','-.','--','-']

In [11]:
# X_train.head().style.background_gradient(cmap='coolwarm')

In [12]:
# K = len(set(y_train))
# y_categ_train = to_categorical(y_train,K)

# x_train1, x_val, y_train1, y_val = train_test_split(x_train, y_categ_train, test_size = 0.1, random_state=42)

In [13]:
model = Sequential()

model.add(Conv2D(32, kernel_size = 3, activation='relu', input_shape = (28, 28, 1)))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(32, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 3, activation='relu'))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size = 5, strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.4))

model.add(Conv2D(128, kernel_size = 4, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.4))
model.add(Dense(10, activation='softmax'))

# COMPILE WITH ADAM OPTIMIZER AND CROSS ENTROPY COST
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [14]:
datagen = ImageDataGenerator(
        rotation_range=10,  
        zoom_range = 0.1,  
        width_shift_range=0.1, 
        height_shift_range=0.1)
        
model.fit_generator(datagen.flow(X_train,Y_train, batch_size=64), epochs = 25, 
    steps_per_epoch = X_train.shape[0]//64, callbacks=[annealer])

<ipython-input-14-8056cf292649>:7: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(X_train,Y_train, batch_size=64), epochs = 25,


Epoch 1/25
656/656 [==============================] - 103s 155ms/step - loss: 0.4396 - accuracy: 0.8654 - lr: 0.0010
Epoch 2/25
656/656 [==============================] - 102s 155ms/step - loss: 0.1283 - accuracy: 0.9612 - lr: 9.5000e-04
Epoch 3/25
656/656 [==============================] - 104s 158ms/step - loss: 0.0931 - accuracy: 0.9710 - lr: 9.0250e-04
Epoch 4/25
656/656 [==============================] - 101s 153ms/step - loss: 0.0808 - accuracy: 0.9744 - lr: 8.5737e-04
Epoch 5/25
656/656 [==============================] - 99s 150ms/step - loss: 0.0707 - accuracy: 0.9790 - lr: 8.1451e-04
Epoch 6/25
656/656 [==============================] - 100s 153ms/step - loss: 0.0623 - accuracy: 0.9816 - lr: 7.7378e-04
Epoch 7/25
656/656 [==============================] - 102s 156ms/step - loss: 0.0598 - accuracy: 0.9824 - lr: 7.3509e-04
Epoch 8/25
656/656 [==============================] - 102s 155ms/step - loss: 0.0545 - accuracy: 0.9838 - lr: 6.9834e-04
Epoch 9/25
656/656 [=================

# Test

In [15]:
import pandas as pd
import keras

In [16]:
x_test = pd.read_csv('test.csv')

x_test = x_test.values.reshape(-1,28,28,1)
x_test = x_test/255

In [17]:
model = keras.models.load_model('model.h5')

In [18]:
y_predict = model.predict(x_test)
y_result=np.argmax(y_predict,axis=1)


In [19]:
submission = pd.Series(y_result, name='Label')
submission = pd.concat([pd.Series(range(1 ,28001) ,name = "ImageId"),   submission], axis = 1)
submission.head(10)


,ImageId,Label
0,1,2
1,2,0
2,3,9
3,4,0
4,5,3
5,6,7
6,7,0
7,8,3
8,9,0
9,10,3


In [20]:
submission.to_csv('submission.csv',index=False)